# Construcción de los modelos de lenguaje

**Integrantes**: Paula Daza, Nicolás Klopstock, Isabella Martínez, Gustavo Mendez. 

Primero, importamos las librerías que vamos a usar en todo el notebook.

In [1]:
import os, re, zipfile, random, pickle
from typing import List, Tuple
from collections import Counter
from tqdm import tqdm

import nltk
from nltk import word_tokenize, sent_tokenize

# Descargar recursos necesarios de NLTK
nltk.download('punkt')  # Tokenizador

# Configuración de NLTK
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## I. Estructuración y organización de datasets

Los datasets utilizados son:

- [20Newsgroups](http://qwone.com/~jason/20Newsgroups/)
- [BAC: The Blog Authorship Corpus](https://huggingface.co/datasets/barilan/blog_authorship_corpus)

Fueron descargados desde el link de Dropbox proporcionado, en el siguiente [enlace](https://huggingface.co/datasets/barilan/blog_authorship_corpus).

Para el siguiente proceso vamos a suponer la siguiente estructura en el directorio.

```
data
│
└───20news-18828
│   └───20news-18828
│       └───alt.atheism
│       └───comp.graphics
│       └───...
│
└───BAC
│   │   5114.male.25.indUnk.Scorpio.xml
│   │   blogs.zip
│
└───EN_Lexicons
│   │   AFINN-111.txt
│   │   senticnet5.py
│   │   SentiWordNet_3.0.0.txt
│   │   WordStat Sentiments.txt
│
└───Multi Domain Sentiment
│   │   negative.review
│   │   processed_acl.tar.gz
│   │   unlabeled.review
```

Nótese que el único paso adicional con respecto a los datos descargados de Dropbox es extraer el archivo `20news-18828.tar.gz`, que da lugar a la carpeta con el mismo nombre.

Vamos a definir las rutas para acceder a los textos en `BAC` y `20N`.

In [2]:
twenty_n_path = os.path.join('data', '20news-18828', '20news-18828')
bac_path = os.path.join('data', 'BAC', 'blogs.zip')

Los textos de `20N` se encuentran en un formato que es conveniente limpiar primero con expresiones `regex`. A continuación, definimos una expresión regular para esto.

In [3]:
twenty_n_regex = r'''
^From:.*?\n|                      # Ignora la línea que empieza con 'From:' y lo que sigue hasta el final de la línea
^Subject:.*?\n|                   # Ignora la línea que empieza con 'Subject:' y lo que sigue hasta el final de la línea
^Archive-name:.*?\n|              # Ignora la línea que empieza con 'Archive-name:' y lo que sigue hasta el final de la línea
^Alt-atheism-archive-name:.*?\n|  # Ignora la línea que empieza con 'Alt-atheism-archive-name:' y lo que sigue hasta el final de la línea
^Last-modified:.*?\n|             # Ignora la línea que empieza con 'Last-modified:' y lo que sigue hasta el final de la línea
^Version:.*?\n|                   # Ignora la línea que empieza con 'Version:' y lo que sigue hasta el final de la línea
^.*@.*?\n|                        # Ignora la línea que contiene '@' y lo que sigue hasta el final de la línea
In\sarticle.*?writes:\n|          # Ignora todo lo que está entre 'In article...' y 'writes:'
[^a-zA-Z0-9\s.,]                  # Elimina cualquier carácter que no sea una letra, un número o un espacio
|^>+                              # Elimina '>' al inicio de una línea
|\s*>+                            # Elimina '>' seguido por espacios
^-+$                              # Ignora las líneas que contienen solo '-'
^=+$                              # Ignora las líneas que contienen solo '='
'''

En la siguiente celda, definimos una función para listar los nombres de los directorios de una ruta en particular, y listar de la misma manera los archivos.

In [4]:
def list_directories(path: str) -> List[str]:
    """
    Retorna una lista de directorios dentro de la ruta especificada.

    Args:
        path (str): Ruta en la que buscar subdirectorios.

    Returns:
        List[str]: Lista de nombres de los subdirectorios.
    """
    return [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]

def list_files(path: str) -> List[str]:
    """
    Retorna una lista de nombres de archivos dentro de la ruta especificada.

    Args:
        path (str): Ruta en la que buscar archivos.

    Returns:
        List[str]: Lista de nombres de archivos
    """
    return [name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))]

Las siguientes funciones obtienen los textos relevantes de `20N` y `BAC`. Nótese que reciben un archivo o la ruta a este, y extraen el texto que nos interesa.

In [5]:
def get_relevant_text_twenty_news(path: str) -> str:
    """
    Extrae y limpia el texto relevante de un archivo utilizando regex.

    Args:
        path (str): La ruta del archivo a leer y procesar.

    Returns:
        str: El texto limpio.
    """
    global twenty_n_regex # Usar una expresión regular global predefinida

    try:
        with open(path, 'r', encoding='utf-8') as file:
            text = file.read()
            # Limpiar el texto utilizando regex
            cleaned_text = re.sub(twenty_n_regex, '', text, flags=re.VERBOSE | re.MULTILINE)
    except (UnboundLocalError, UnicodeDecodeError) as e:
        return ''


    # Eliminar espacios en blanco repetidos y retornar el texto
    return ' '.join(cleaned_text.split())

def get_relevant_text_bac(content: str) -> List[Tuple[str, str]]:
    """
    Extrae y limpia el texto relevante de datos BAC, incluyendo fechas y el texto en cuestión (las publicaciones).

    Args:
        content (str): El contenido del texto a procesar.

    Returns:
        List[Tuple[str, str]]: Una lista de tuplas donde cada tupla contiene una fecha y su texto.
    """
    # Extraer todas las fechas y publicaciones del contenido
    dates = re.findall(r'<date>(.*?)</date>', content, re.DOTALL)
    posts = re.findall(r'<post>(.*?)</post>', content, re.DOTALL)
    
    # Limpiar los textos eliminando espacios innecesarios y palabras irrelevantes
    cleaned_posts = [' '.join(re.sub(r'\s+', ' ', post).split()).replace('urlLink', '') for post in posts]
    
    # Combinar las fechas y los textos en tuplas y retornarlas
    return list(zip(dates, cleaned_posts))

Las siguientes son las funciones de procesamiento principales para consolidar todos los archivos tanto de `BAC` como de `20N` en sus propios `txt` aparte. Nótese que generan archivos llamados `consolidated_bac.txt` y `consolidated_news.txt` en el mismo directorio en donde se ejecute este notebook.

In [6]:
def process_twenty_news_data(path: str) -> None:
    """
    Procesa todos los datos de 20N, consolida el texto y lo escribe en un archivo.

    Args:
        ruta (str): La ruta del directorio que contiene los datos de noticias.
    """
    with open('consolidated_news.txt', 'w', encoding='utf-8') as output_file:

        # Listar los subdirectorios que contienen las noticias
        news_dirs = list_directories(path)
        
        # Para cada subdirectorio
        for each_news_dir in news_dirs:
            each_path = os.path.join(path, each_news_dir) # Ruta completa del subdirectorio
            news_files = list_files(each_path) # Listar archivos dentro del subdirectorio
            
            # Para cada archivo de noticias
            for each_news_file in news_files:
                file_path = os.path.join(each_path, each_news_file) # Ruta completa del archivo
                text = get_relevant_text_twenty_news(file_path) # Extraer el texto relevante
                if text:
                    output_file.write(text + '\n') # Escribir el texto en el archivo de salida

def process_bac_data(zip_path: str) -> None:
    """
    Procesa todos los datos de BAC desde un archivo zip, extrae información relevante y la escribe en un archivo.

    Args:
        ruta_zip (str): La ruta al archivo zip que contiene los datos de BAC.
    """
    # Abrir el archivo zip para lectura
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_names = zip_ref.namelist() # Obtener la lista de nombres de archivos en el zip
        
        with open('consolidated_bac.txt', 'w', encoding='utf-8') as output_file:
            # Para cada archivo en el zip
            for file_name in file_names:
                if not file_name.endswith('/'): # Ignorar directorios dentro del zip
                    with zip_ref.open(file_name) as file:
                        try:
                            # Intentar leer el contenido del archivo como UTF-8
                            content = file.read().decode('utf-8')
                        except UnicodeDecodeError:
                            # Si falla, leerlo como latin1
                            content = file.read().decode('latin1')
                        
                        # Extraer las parejas de fecha y texto
                        data_pairs = get_relevant_text_bac(content)
                        
                        # Escribir cada pareja en el archivo de salida
                        for date, post in data_pairs:
                            output_file.write(f"{date} - {post}\n")

Finalmente, la siguiente celda ejecuta estos procesos y genera los archivos consolidados.

In [7]:
process_twenty_news_data(twenty_n_path)
process_bac_data(bac_path)

## II. Preprocesamiento de texto (tokenización, tokens especiales, normalización)

In [8]:
def preprocess_text(file_path: str) -> List[str]:
    """
    Procesa el texto de la ruta especificada, aplicando lo siguiente:

    - Tokenización por oración
    - Normalización
    - Reemplaza números por NUM
    - Añade tokens de inicio y finalización de oración <s> y </s>
    - Tokens con frecuencia unitaria se marcan como <UNK>

    Args:
        path (str): Ruta en la que leer el archivo de texto.

    Returns:
        List[str]: Listado de oraciones preprocesadas.
    """
    # Leer el archivo de texto línea por línea
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    sentences = []
    for line in lines:
        # Dividir cada línea en oraciones
        line_sentences = sent_tokenize(line.strip())
        sentences.extend(line_sentences)
    
    # Listas donde guardar tokens y oraciones procesadas
    all_tokens = []
    processed_sentences = []
    
    # Para cada oración...
    for sentence in tqdm(sentences, desc="Procesando oraciones..."):

        # Pasar a minúsculas
        sentence = sentence.lower()
        
        # Separar números de letras (e.g. 9am -> 9 am)
        sentence = re.sub(r'(\d+)([a-zA-Z])', r'\1 \2', sentence)
        sentence = re.sub(r'([a-zA-Z])(\d+)', r'\1 \2', sentence)
        
        # Reemplazar números por el token NUM
        sentence = re.sub(r'\d+', 'NUM', sentence)
        
        # Eliminar puntuación y caracteres especiales
        sentence = re.sub(r'[^\w\s]', ' ', sentence)
        
        # Tokenizar palabras con nltk
        tokens = word_tokenize(sentence)
        
        # Añadir tokens de inicio y finalización
        tokens = ['<s>'] + tokens + ['</s>']
        
        # Agregar a las listas predefinidas si no es una oración vacía
        if len(tokens) > 2:
            all_tokens.extend(tokens)
            processed_sentences.append(tokens)
    
    # Contar la frecuencia de los tokens
    token_freq = Counter(all_tokens)
    
    # Reemplazar tokens de frecuencia unitaria con <UNK>
    final_sentences = []
    for tokens in processed_sentences:
        final_tokens = [token if token_freq[token] > 1 else '<UNK>' for token in tokens]
        final_sentences.append(final_tokens)
    
    return final_sentences

In [9]:
final_sentences_news = preprocess_text('consolidated_news.txt')
with open('processed_news.txt', 'w', encoding='utf-8') as output_file:
    for sentence in final_sentences_news:
        output_file.write(' '.join(sentence) + '\n')

Procesando oraciones...: 100%|██████████| 240621/240621 [00:17<00:00, 13849.94it/s]


In [10]:
final_sentences_bac = preprocess_text('consolidated_bac.txt')
with open('processed_bac.txt', 'w', encoding='utf-8') as output_file:
    for sentence in final_sentences_bac:
        output_file.write(' '.join(sentence) + '\n')

Procesando oraciones...: 100%|██████████| 7144331/7144331 [07:47<00:00, 15266.19it/s]


## III. Creación de conjuntos de entrenamiento y evaluación 

In [11]:
def generate_training_testing(final_sentences: List[str], filename: str) -> None:
    """
    Genera archivos de texto que contienen conjuntos de oraciones para entrenamiento y prueba, en una
    proporción de 80-20.

    Args:
        final_sentences (List[str]): Lista de oraciones que se usarán para generar los conjuntos.
        filename (str): Nombre base del archivo de salida, sin extensión. 
                        Se crearán dos archivos: `<filename>_training.txt` y `<filename>_testing.txt`.
    """
    # Hacer una copia de las oraciones y mezclarlas aleatoriamente
    sents = final_sentences.copy()
    random.shuffle(sents)

    # Determinar el punto de separación para 80-20
    split_index = int(0.8 * len(sents))

    # Separar en conjuntos de entrenamiento y prueba
    training_sentences = sents[:split_index]
    testing_sentences = sents[split_index:]

    # Guardar las oraciones de entrenamiento en un archivo txt
    with open(f'{filename}_training.txt', 'w', encoding='utf-8') as training_file:
        for sentence in training_sentences:
            training_file.write(' '.join(sentence) + '\n')

    # Guardar las oraciones de prueba en un archivo txt
    with open(f'{filename}_testing.txt', 'w', encoding='utf-8') as testing_file:
        for sentence in testing_sentences:
            testing_file.write(' '.join(sentence) + '\n')

In [12]:
generate_training_testing(final_sentences_news, "20N")
generate_training_testing(final_sentences_bac, "BAC")

## IV. Creación de los modelos de lenguaje

En este punto es buena idea reiniciar el kernel y liberar memoria RAM.

In [1]:
import pickle
from typing import List, Dict, Tuple
from collections import Counter
from tqdm import tqdm

import nltk

# Descargar recursos necesarios de NLTK
nltk.download('punkt') 

# Configuración de NLTK
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
def read_and_tokenize(file_path: str) -> List[List[str]]:
    """
    Lee un archivo de texto, tokeniza las oraciones y retorna una lista de listas de tokens.
    
    Args:
        file_path: Ruta al archivo de texto.
        
    Returns:
        List[List[str]]: Lista de listas de tokens (cada oración tokenizada).
    """
    with open(file_path, 'r', encoding="utf-8") as f:
        text = f.read()

    sentences = text.strip().split('\n')
    tokenized_sentences = [sentence.split() for sentence in sentences]
    return tokenized_sentences


def build_ngram_counts(tokenized_sentences: List[List[str]], n: int) -> Counter:
    """
    Cuenta los n-gramas de las oraciones tokenizadas usando nltk
    
    Args:
        tokenized_sentences: Lista de listas de tokens.
        n: Tamaño del n-grama.
        
    Returns:
        Counter: Un diccionario con los conteos de n-gramas.
    """
    ngram_counts = Counter()
    
    for sentence in tokenized_sentences:
        ngrams = list(nltk.ngrams(sentence, n))
        ngram_counts.update(ngrams)
    
    return ngram_counts


def calculate_ngram_probabilities(tokenized_sentences: List[List[str]], n: int) -> Dict[Tuple[str, ...], float]:
    """
    Calcula las probabilidades de n-gramas con suavizado de Laplace.
    
    Args:
        tokenized_sentences: Lista de listas de tokens.
        n: Tamaño del n-grama.
        
    Returns:
        Dict[Tuple[str, ...], float]: Diccionario con las probabilidades de los n-gramas.
    """
    # Cuenta los n-gramas y de ser necesario los (n-1)-gramas, para bigramas y trigramas.
    ngram_counts = build_ngram_counts(tokenized_sentences, n)
    n_minus_1_gram_counts = build_ngram_counts(tokenized_sentences, n-1) if n > 1 else None

    # Tamaño del vocabulario
    vocabulary = set(token for sentence in tokenized_sentences for token in sentence)
    vocab_size = len(vocabulary)

    # Diccionario que representará nuestro modelo
    ngram_probabilities = {}

    for ngram, count in tqdm(ngram_counts.items(), desc=f"Calculando {n}-gramas"):
        if n == 1:
            # Para unigramas el conteo es sencillo
            total_count = sum(ngram_counts.values())
            ngram_probabilities[ngram] = (count + 1) / (total_count + vocab_size)
        else:
            # Para bigramas y trigramas hay que tener en cuenta el contexto
            priori = ngram[:-1]
            priori_count = n_minus_1_gram_counts[priori]
            ngram_probabilities[ngram] = (count + 1) / (priori_count + vocab_size)
    
    return ngram_probabilities

def save_ngram_model(ngram_probabilities: Dict[Tuple[str, ...], float], filename: str) -> None:
    """
    Guarda el modelo de lenguaje de n-gramas en un archivo binario usando pickle.
    
    Args:
        ngram_probabilities (Dict[Tuple[str, ...], float]): Diccionario con las probabilidades de los n-gramas.
        filename (str): Nombre del archivo donde se guardará el modelo.
    """
    with open(filename, 'wb') as file:
        pickle.dump(ngram_probabilities, file)

### 20 News

In [14]:
tokenized_sentences = read_and_tokenize('20N_training.txt')

unigram_probs = calculate_ngram_probabilities(tokenized_sentences, 1)
bigram_probs = calculate_ngram_probabilities(tokenized_sentences, 2)
trigram_probs = calculate_ngram_probabilities(tokenized_sentences, 3)

Calculando 3-gramas: 100%|██████████| 1991606/1991606 [00:01<00:00, 1045402.69it/s]


In [15]:
print("Unigrams:")
for unigram, prob in list(unigram_probs.items())[:5]:
    print(f"{unigram}: {prob}")

print("\nBigrams:")
for bigram, prob in list(bigram_probs.items())[:5]:
    print(f"{bigram}: {prob}")

print("\nTrigrams:")
for trigram, prob in list(trigram_probs.items())[:5]:
    print(f"{trigram}: {prob}")

Unigrams:
('<s>',): 0.045005262235729766
('ive',): 0.0006062831306847886
('seen',): 0.00031693680577221157
('him',): 0.000640712277740401
('play',): 0.00022638343269443684

Bigrams:
('<s>', 'ive'): 0.0037389436089176596
('ive', 'seen'): 0.004984423676012461
('seen', 'him'): 0.00019554165037152912
('him', 'play'): 0.00022313957380341404
('play', 'for'): 0.0005083300537845993

Trigrams:
('<s>', 'ive', 'seen'): 0.001345100226370526
('ive', 'seen', 'him'): 3.314770617873243e-05
('seen', 'him', 'play'): 6.66266906522753e-05
('him', 'play', 'for'): 3.331223558413005e-05
('play', 'for', 'the'): 0.00026642244609108317


In [16]:
save_ngram_model(unigram_probs, "20N_unigrams.pkl")
save_ngram_model(bigram_probs, "20N_bigrams.pkl")
save_ngram_model(trigram_probs, "20N_trigrams.pkl")

### BAC

In [3]:
tokenized_sentences_bac = read_and_tokenize('BAC_training.txt')

unigram_probs_bac = calculate_ngram_probabilities(tokenized_sentences_bac, 1)
bigram_probs_bac = calculate_ngram_probabilities(tokenized_sentences_bac, 2)
trigram_probs_bac = calculate_ngram_probabilities(tokenized_sentences_bac, 3)

Calculando 3-gramas: 100%|██████████| 31428880/31428880 [00:31<00:00, 1003459.65it/s]


In [4]:
print("Unigrams:")
for unigram, prob in list(unigram_probs_bac.items())[:5]:
    print(f"{unigram}: {prob}")

print("\nBigrams:")
for bigram, prob in list(bigram_probs_bac.items())[:5]:
    print(f"{bigram}: {prob}")

print("\nTrigrams:")
for trigram, prob in list(trigram_probs_bac.items())[:5]:
    print(f"{trigram}: {prob}")

Unigrams:
('<s>',): 0.05508172833221783
('i',): 0.0338773017666548
('ll',): 0.0012488989901693823
('kill',): 0.00011111598484642159
('you',): 0.008370967397909566

Bigrams:
('<s>', 'i'): 0.15295956856258663
('i', 'll'): 0.020992896480166015
('ll', 'kill'): 0.00053326390053752
('kill', 'you'): 0.0028410902418114963
('you', 'when'): 0.0013790266120353178

Trigrams:
('<s>', 'i', 'll'): 0.015410658445388373
('i', 'll', 'kill'): 0.0003745511105011551
('ll', 'kill', 'you'): 0.0002545571259393711
('kill', 'you', 'when'): 1.1043703616444811e-05
('you', 'when', 'you'): 0.0024852427241490705


In [5]:
save_ngram_model(unigram_probs_bac, "BAC_unigrams.pkl")
save_ngram_model(bigram_probs_bac, "BAC_bigrams.pkl")
save_ngram_model(trigram_probs_bac, "BAC_trigrams.pkl")